## Silver Data Transformation

In [0]:
df = spark.read\
    .format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load("abfss://raw@netflixprojectsa10.dfs.core.windows.net/netflix_titles.csv")

In [0]:
df.display()

In [0]:
df.columns

In [0]:
df=df.fillna({"duration_minutes":0,"duration_seasons":1})

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import col,split,when
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df=df.withColumn("duration_minutes",col("duration_minutes").cast("int"))\
    .withColumn("duration_seasons",col("duration_seasons").cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
df=df.withColumn("title",split(col("title"),':')[0])

In [0]:
df=df.withColumn("rating",split(col("rating"),"-")[0])
df.display()

In [0]:
df=df.withColumn("type_flag",when(col("type")=="Movie",1).\
    when(col("type")=="TV Show",2).otherwise(0))

In [0]:
df.display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank,count

In [0]:
df=df.withColumn("duration_dense_ranked",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
df.display()

In [0]:
vis_df=df.groupBy("type").agg(count("*").alias("total_count"))

In [0]:
vis_df.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.mode("overwrite").format("delta").option("path","abfss://silver@netflixprojectsa10.dfs.core.windows.net/netflix_titles").save()